In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark import SparkContext
import requests
from pyspark.sql import functions as F
from pyspark.sql import types

sc = SparkContext()
spark = SparkSession.builder \
    .appName('test') \
    .getOrCreate()



23/05/07 16:04:25 WARN Utils: Your hostname, dell resolves to a loopback address: 127.0.1.1; using fd01:db8:1111:0:0:0:0:2 instead (on interface lo)
23/05/07 16:04:25 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/05/07 16:04:25 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [ ]:

symbol = ['AAPL', 'MSFT', 'GOOG', 'AMZN', 'TSLA', 'META', 'NFLX', 'NVDA']

url = 'https://api.twelvedata.com/time_series'

schema = types.StructType([
    types.StructField("close", types.FloatType(), True),
    types.StructField("high", types.FloatType(), True),
    types.StructField("loew", types.FloatType(), True),
    types.StructField("open", types.FloatType(), True),
    types.StructField("volume", types.IntegerType(), True),
    types.StructField("symbol", types.StringType(), True),
    types.StructField("datetime", types.StringType(), True)
])

for sym in symbol:
    params = {'symbol': sym, 'interval': '1h','date':'2019-01-11', 'apikey': '2efb4e0c80a041b794abaf4369e76869', 'timezone':'America/New_York'}

    response = requests.get(url, params=params)

    if response.status_code == requests.codes.ok:
        data = response.json()
        print(data)
        df = spark.read.json(sc.parallelize([data['values']]))
        df = df.withColumn("symbol", F.lit(sym))
        
        df = df.withColumn("datetime", F.to_timestamp("datetime", "yyyy-MM-dd HH:mm:ss"))
        df = df.withColumn("close", df["close"].cast(types.FloatType()))
        df = df.withColumn("high", df["high"].cast(types.FloatType()))
        df = df.withColumn("low", df["low"].cast(types.FloatType()))
        df = df.withColumn("open", df["open"].cast(types.FloatType()))
        df = df.withColumn("volume", df["volume"].cast(types.IntegerType()))
        df = df.withColumn("symbol", df["symbol"].cast(types.StringType()))
        df = df.withColumn("datetime", df["datetime"].cast(types.TimestampType()))


#         df = spark.createDataFrame(df.rdd, schema=schema)
        print(df.schema)
        df.show()
        # Write the data to a Parquet file partitioned by date
        df.coalesce(1).write.mode("append").parquet("stock_data/2019/01")
    else:
        print('Error:', response.status_code, response.text)
